In [47]:
import numpy as np
import os
import torch
from torch import nn
from d2l import torch as d2l


# Prepare pretrained embedding dl

We use GloVe, a stable version of word2vec. (see detailed explanations [here](https://d2l.ai/chapter_natural-language-processing-pretraining/glove.html)) or [fastText](https://fasttext.cc/)

In [48]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [49]:
d2l.DATA_HUB['wiki.en']

('http://d2l-data.s3-accelerate.amazonaws.com/wiki.en.zip',
 'c1816da3821ae9f43899be655002f6c723e91b88')

# Load embedding through a class

In [50]:
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [
            self.token_to_idx.get(token, self.unknown_idx)
            for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [51]:
glove_6b50d = TokenEmbedding('glove.6b.50d')


In [52]:
# General infos
print(len(glove_6b50d))

# Attributes
print('tokens:', glove_6b50d.idx_to_token[:3])
print('vecs:', glove_6b50d.idx_to_vec.shape)
print('idx:', list(glove_6b50d.token_to_idx.items())[:3])

# Dict {token:vector}
glove_6b50d['a'].shape

400001
tokens: ['<unk>', 'the', ',']
vecs: torch.Size([400001, 50])
idx: [('<unk>', 0), ('the', 1), (',', 2)]


torch.Size([1, 50])

## Synonyms with...
K nearest neirghbor

In [104]:
def knn(W, x, k):
    cos_sim = torch.mv(W, x.reshape(-1,)) / (torch.sqrt(torch.sum(W* W, axis=1) + 1e-9 ) * torch.sqrt((x*x).sum()))
    topk_val, topk_idx = torch.topk(cos_sim, k=k)
    topk_val = [cos_sim[int(i)] for i in topk_idx]
    return topk_idx , topk_val


In [105]:
a = glove_6b50d['a'] * glove_6b50d['a']
torch.sum(a, axis=1)
a.reshape(-1,).shape, glove_6b50d['a'].shape

(torch.Size([50]), torch.Size([1, 50]))

In [108]:
def get_similar_tokens(query_token, k, embed):
    topk_idx, topk_val = knn(embed.idx_to_vec, embed[[query_token]], k+2)
    for i, c in zip(topk_idx[2:], topk_val[2:]): # remove input words
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

    

get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.749: intel
cosine sim=0.749: electronics
cosine sim=0.731: semiconductor


In [109]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.800: boy
cosine sim=0.792: girl
cosine sim=0.778: newborn


In [110]:
get_similar_tokens('beautiful', 3, glove_6b50d)


cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful
cosine sim=0.825: charming


### Analogies
Analogy of man is woman and of son is daughter.

This relation is formally written as: “man”:“woman”::“son”:“daughter”
We can generalize to:
``a:b::c:d`` 
Where we look for d, given a,b,c.

We can find d with the formula:
``vec(c)+vec(b)-vec(a)``

In [111]:
def get_analogy(token_a, token_b, token_c, embed):
    vec_d = embed[[token_c]] + \
            embed[[token_b]] - \
            embed[[token_a]]
    topk, cos = knn(embed.idx_to_vec, vec_d, 1)
    return embed.idx_to_token[int(topk[0])]

In [112]:
get_analogy('man', 'woman', 'son', glove_6b50d)


'daughter'

In [113]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)


'japan'

In [114]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [115]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'